In [1]:
import os


In [2]:
%pwd

'/Users/swatip1705/Documents/Projects/Swayam/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/swatip1705/Documents/Projects/Swayam'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    


In [8]:
from Swayam_TextSummarization.constants import *
from Swayam_TextSummarization.utils.common import read_yaml, create_directories


In [32]:
class ConfigurationManager:
    def __init__(self, config_filepath: Path= CONFIG_FILE_PATH, params_filepath: Path= PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir



        )
        return data_ingestion_config

In [33]:
import os
import urllib.request as request
import zipfile
from Swayam_TextSummarization.logging import logger
from Swayam_TextSummarization.utils.common import get_size

In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self): 
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file

            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
            
        else:
            
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [36]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:  
    raise e

[2025-04-04 22:12:57,825]:INFO:common:Successfully loaded YAML file: config/config.yaml]
[2025-04-04 22:12:57,829]:INFO:common:Successfully loaded YAML file: params.yaml]
[2025-04-04 22:12:57,831]:INFO:common:Created directory: artifacts]
[2025-04-04 22:12:57,832]:INFO:common:Created directory: artifacts/data_ingestion]
[2025-04-04 22:12:59,605]:INFO:2424851550:artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F427:090C:180A1:3A718:67F00C12
Accept-Ranges: bytes
Date: Fri, 04 Apr 2025 16:42:59 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10224-MAA
X-Cache: MISS
X-Cache-Hits: 0